In [1]:
!pip install datasets
from datasets import load_dataset
import datasets
import torch
#!pip install torch==1.7.1
!pip install transformers
!pip install sentencepiece 
from torch.utils.data import DataLoader
from  transformers.data.data_collator import default_data_collator

In [2]:
import os 
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/pl")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from transformers import AutoTokenizer

In [4]:
import pandas as pd
#test_data = pd.read_csv('/content/drive/MyDrive/pl/testData.csv')
train_data = pd.read_csv('/content/drive/MyDrive/pl/trainData.csv')
#valid_data= pd.read_csv('/content/drive/MyDrive/pl/valData.csv')

In [5]:
#test_data = test_data.drop('Unnamed: 0' ,axis = 1)
train_data = train_data.drop('Unnamed: 0' ,axis = 1)
#valid_data = valid_data.drop('Unnamed: 0' ,axis = 1)

In [6]:
#test_data = test_data.to_dict(orient='list')
train_data = train_data.to_dict(orient='list')
#valid_data = valid_data.to_dict(orient='list')


In [7]:
from datasets import Dataset
train_data = Dataset.from_dict(train_data)
#test_data = Dataset.from_dict(test_data)
#valid_data = Dataset.from_dict(valid_data)
import gc
gc.collect()

0

In [8]:
train_data = Dataset.from_dict(train_data[136000:])
import gc
gc.collect()

0

we choose 1000 cause if we choose more we can't use batch size = 8 

we choose xsum model after a comaprasion bet. xsum and pegasus paraphase because of the pl labels we have

we change the config of the google/pegasus/xsum to change the position_embd to 1024

we can't use more than 68000example at a time cause we were using colab and the ram is crashed if we use more

In [10]:
#tokenizer = AutoTokenizer.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")
#test_dataset_document = tokenizer(test_data['document'],truncation=True,padding='longest',max_length = 1024)
#test_dataset_summary = tokenizer(test_data['summary'],truncation=True,padding='longest',max_length = 1024)
train_data_document = tokenizer(train_data['document'],truncation=True,padding='longest',max_length = 1000,)#return_tensors="pt").to("cuda")
train_data_summary = tokenizer(train_data['summary'],truncation=True,padding='longest',max_length = 60,)#return_tensors="pt").to("cuda")
#valid_data_document = tokenizer(valid_data['document'],truncation=True,padding='longest',max_length = 1000,return_tensors="pt").to("cuda")
#valid_data_summary = tokenizer(valid_data['summary'],truncation=True,padding='longest',max_length = 60,return_tensors="pt").to("cuda")
gc.collect()

0

In [11]:
import gc
gc.collect()

44

In [12]:
train_data_document = Dataset.from_dict(train_data_document)
train_data_summary = Dataset.from_dict(train_data_summary)
#valid_data_document = Dataset.from_dict(dict(**valid_data_document))
#valid_data_summary = Dataset.from_dict(dict(**valid_data_summary))

In [13]:
train_data_summary = train_data_summary.rename_column('attention_mask','decode_mask')
train_data_summary = train_data_summary.rename_column('input_ids','decode_ids')
#valid_data_summary = valid_data_summary.rename_column('attention_mask','decode_mask')
#valid_data_summary = valid_data_summary.rename_column('input_ids','decode_ids')

In [14]:
train_data = Dataset.from_dict(dict(**train_data_document[:],**train_data_summary[:]))
#

In [18]:
#valid_data = Dataset.from_dict(dict(**valid_data_document[:],**valid_data_summary[:]))

In [15]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')
#valid_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [15]:
train_data['input_ids'][0].is_cuda

True

In [16]:
train_data.save_to_disk('/content/drive/MyDrive/pl/data/xsumtrainDataset_3')
#valid_data.save_to_disk('/content/drive/MyDrive/pl/data/xsumvalidDataset')

In [31]:
train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_1")
valid_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumvalidDataset")

In [38]:
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [39]:
train_dataset['input_ids'][0:1].is_cuda

True

In [3]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [4]:
model2 = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum",max_length = 1000,max_position_embeddings=1000).to('cuda')

In [2]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model2 = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum",max_length = 1000,max_position_embeddings=1000).to('cuda')
torch.save(model2,'/content/drive/MyDrive/teacher_model.pt')

In [6]:
model2.config

PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,
